**Unzip the uploaded data files**

In [7]:
# !unzip "drive/MyDrive/Dog Vision/dog-breed-identification.zip" -d "drive/MyDrive/Dog Vision/"

# End-to-end multi-class Dog Breed Classification
This notebook builds an end-to-end multi-class image classifier using TensorFlow 2.5.0 and TensorFlow Hub.

## 1. Problem
Identifing the breed of dog using a given image of the dog.
When I'm sitting in a cafe and I take a photo of a dog, I want to know what breed of dog it is.

## 2. Data
The data that we are using is from Kaggle's Dog Breed Identification competition.
https://www.kaggle.com/c/dog-breed-identification/data

## 3. Evaluation
Submissions are evaluated on Multi Class Log Loss between the predicted probability and the observed target.
https://www.kaggle.com/c/dog-breed-identification/overview/evaluation

For each image in the test set, you must predict a probability for each of the different breeds. The file should contain a header and have the following format:

id,affenpinscher,afghan_hound,..,yorkshire_terrier
000621fb3cbb32d8935728e48679680e,0.0083,0.0,...,0.0083
etc.

## 4. Features
Some information about the data:
* We're dealing with images (unstructured data), so it's probably best we use Deep Learning / Transfer Learning.
* There are 120 breeds of dogs (This means there are 120 different classes.
* There are around 10,000+ images in the training set (These images have labels).
* There around 10,000+ images in the test set (these images will have no labels, because we want to predict them).



# Get our workspace ready

* Import TensorFLow
* Import TensorFlow Hub
* Make sure we'e using a GPU

In [3]:
# Import TensorFlow in Colab
import tensorflow as tf
print("TensorFlow Version: " + tf.__version__)

TensorFlow Version: 2.5.0


In [4]:
# Import TensorFlow Hub
import tensorflow_hub as hub
print("TensorFlow Hub Version: " + hub.__version__)

TensorFlow Hub Version: 0.12.0


In [5]:
# Check for GPU availability
# https://research.google.com/colaboratory/faq.html#gpu-availability
print("GPU", " available (YESS!!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

GPU  available (YESS!!!!!)


In [6]:
# Change Runtime Type to GPU
# Go to Runtime -> Chage Runtime Type -> Select GPU and save